In [1]:
!pip install transformers rouge_score -q

In [18]:
import pandas as pd
test_news = pd.read_csv('/kaggle/input/news-summary/test_news.csv')
test_news = test_news[['summary', 'news']]
test_news

,summary,news
0,the indian space research organisation isro is...,summarize: the indian space research organisat...
1,women and children will be allowed to use toil...,summarize: starting monday women and children ...
2,an image of a pregnant us president donald tru...,summarize: the image of russian president vlad...
3,banking operations across the country came to ...,summarize: banking operations across the count...
4,indian national congress could manage to win o...,summarize: six out of 10 assembly seats in ame...
...,...,...
387,sasi gangadharan a semiparalysed man from kera...,summarize: eighteen years ago sasi gangadharan...
388,usbased researchers have identified that the h...,summarize: washington jan 10 pti the human app...
389,aap mla kapil mishra who was sacked on saturda...,summarize: rumblings in the aam aadmi party co...
390,a child rights commission in tamil nadu has re...,summarize: after a complaint filed by a child ...


In [5]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

## Load Pretrained T5 Small Model and Tokenizer

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
t5_small_model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)
t5_small_tokenizer = T5Tokenizer.from_pretrained('t5-small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Load Pretrained T5 Base Model and Tokenizer

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
t5_base_model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)
t5_base_tokenizer = T5Tokenizer.from_pretrained('t5-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Load Pretrained BART Base Model and Tokenizer

In [8]:
from transformers import BartForConditionalGeneration, BartTokenizer
bart_base_model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)
bart_base_tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

## Load Pretrained BART Large Model and Tokenizer

In [9]:
from transformers import BartForConditionalGeneration, BartTokenizer
bart_large_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)
bart_large_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

## Create Pipeline

In [10]:
from transformers import pipeline

2024-04-01 06:13:04.692510: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 06:13:04.692610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 06:13:04.818772: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
t5_small_pipe = pipeline("summarization",
                         model = t5_small_model,
                         tokenizer = t5_small_tokenizer,
                         device=1)

In [12]:
t5_base_pipe = pipeline("summarization",
                       model = t5_base_model,
                       tokenizer = t5_base_tokenizer,
                       device = 1)

In [13]:
bart_base_pipe = pipeline("summarization",
                         model = bart_base_model,
                         tokenizer = bart_base_tokenizer,
                         device = 1)

In [23]:
bart_large_pipe = pipeline("summarization",
                           model = bart_large_model,
                           tokenizer = bart_large_tokenizer,
                           device = 1)

## Defining Function to Generate Summary

In [15]:
def generate_summary(pipe,text):
    summary = pipe(text)
    return summary[0]['summary_text']

In [19]:
test_news['t5_small_pretrained'] = test_news['news'].apply(lambda x: generate_summary(t5_small_pipe, x))

Your max_length is set to 200, but your input_length is only 197. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=98)
Your max_length is set to 200, but your input_length is only 115. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
Token indices sequence length is longer than the specified maximum sequence length for this model (550 > 512). Running this sequence through the model will result in indexing errors
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Your max_length is set to 200, but your input_length is only 170. Since this is a summarization task, 

In [20]:
test_news['t5_base_pretrained'] = test_news['news'].apply(lambda x: generate_summary(t5_base_pipe, x))

Your max_length is set to 200, but your input_length is only 197. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=98)
Your max_length is set to 200, but your input_length is only 115. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Your max_length is set to 200, but your input_length is only 170. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=85)
Your max_length is set to 200, but

In [21]:
test_news['bart_base_pretrained'] = test_news['news'].apply(lambda x: generate_summary(bart_base_pipe, x))

Your max_length is set to 128, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Your max_length is set to 128, but your input_length is only 112. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)
Your max_length is set to 128, but your input_length is only 60. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)
Your max_length is set to 128, but y

In [24]:
test_news['bart_large_pretrained'] = test_news['news'].apply(lambda x: generate_summary(bart_large_pipe, x))

Your max_length is set to 142, but your input_length is only 92. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Your max_length is set to 142, but your input_length is only 137. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=68)
Your max_length is set to 142, but your input_length is only 132. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 142, but 

## Evaluation

In [25]:
from rouge_score import rouge_scorer
import numpy as np

def evaluate_test_summaries(news_data,model_name):
    scorer = rouge_scorer.RougeScorer(rouge_types=['rouge1', 'rouge2', 'rougeL'])
    rouge_scores = []

    for idx, row in test_news.iterrows():
        scores = scorer.score(target=row['summary'], prediction=row[model_name])
        rouge_scores.append(scores)

    avg_rouge1_precision = np.mean([score['rouge1'].precision for score in rouge_scores])
    avg_rouge1_recall = np.mean([score['rouge1'].recall for score in rouge_scores])
    avg_rouge1_f1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])

    avg_rouge2_precision = np.mean([score['rouge2'].precision for score in rouge_scores])
    avg_rouge2_recall = np.mean([score['rouge2'].recall for score in rouge_scores])
    avg_rouge2_f1 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])

    avg_rougeL_precision = np.mean([score['rougeL'].precision for score in rouge_scores])
    avg_rougeL_recall = np.mean([score['rougeL'].recall for score in rouge_scores])
    avg_rougeL_f1 = np.mean([score['rougeL'].fmeasure for score in rouge_scores])

    print("Average ROUGE-1 Precision: ", avg_rouge1_precision)
    print("Average ROUGE-1 Recall: ", avg_rouge1_recall)
    print("Average ROUGE-1 F1-Score: ", avg_rouge1_f1)
    print("Average ROUGE-2 Precision: ", avg_rouge2_precision)
    print("Average ROUGE-2 Recall: ", avg_rouge2_recall)
    print("Average ROUGE-2 F1-Score: ", avg_rouge2_f1)
    print("Average ROUGE-L Precision: ", avg_rougeL_precision)
    print("Average ROUGE-L Recall: ", avg_rougeL_recall)
    print("Average ROUGE-L F1-Score: ", avg_rougeL_f1)

In [26]:
print("For T5-Small (Pretrained):- ")
print("\nEvaluation for the test summary: \n")
evaluate_test_summaries(test_news, 't5_small_pretrained')
print("--"*15)
print("\nFor T5-Base (Pretrained):- ")
print("\nEvaluation for the test summary: \n")
evaluate_test_summaries(test_news, 't5_base_pretrained')
print("--"*15)
print("\nFor BART-Base (Pretrained):- ")
print("\nEvaluation for the test summary: \n")
evaluate_test_summaries(test_news, 'bart_base_pretrained')
print("--"*15)
print("\nFor BART-Large (Pretrained):- ")
print("\nEvaluation for the test summary: \n")
evaluate_test_summaries(test_news, 'bart_large_pretrained')

For T5-Small (Pretrained):- 

Evaluation for the test summary: 

Average ROUGE-1 Precision:  0.49655667925059294
Average ROUGE-1 Recall:  0.38876061595534966
Average ROUGE-1 F1-Score:  0.4260161559755122
Average ROUGE-2 Precision:  0.2549825182307707
Average ROUGE-2 Recall:  0.19879065699751555
Average ROUGE-2 F1-Score:  0.21780845103492702
Average ROUGE-L Precision:  0.3705391909866676
Average ROUGE-L Recall:  0.2883944990299407
Average ROUGE-L F1-Score:  0.31649277875429177
------------------------------

For T5-Base (Pretrained):- 

Evaluation for the test summary: 

Average ROUGE-1 Precision:  0.4695786821463848
Average ROUGE-1 Recall:  0.4155796946079179
Average ROUGE-1 F1-Score:  0.43201966686699794
Average ROUGE-2 Precision:  0.23372603388779198
Average ROUGE-2 Recall:  0.20668007523945398
Average ROUGE-2 F1-Score:  0.21427308628140798
Average ROUGE-L Precision:  0.3436499167526626
Average ROUGE-L Recall:  0.30255407707916077
Average ROUGE-L F1-Score:  0.3148852736743317
-------